In [1]:
import os
import numpy as np 
import pandas as pd 

In [3]:
!wget -P ./../../../dataset/ -nc https://cleanlab-public.s3.amazonaws.com/RegressionBenchmark/airquality_co.csv

File './../../../dataset/airquality_co.csv' already there; not retrieving.



In [4]:
path = "./../../../dataset/airquality_co.csv"
dataset_name = os.path.splitext(os.path.basename(path))[0]
data = pd.read_csv(path, index_col=0)
print("shape of data: ", data.shape)
data.head() 

shape of data:  (7344, 12)


,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),true_label,given_label,true_error
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1692.0,1268.0,1.2078,1.2366,0
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,1559.0,972.0,0.9290,1.1747,0
2,evening,wednesday,11.9,54.0,0.7502,939.0,1140.0,1555.0,1074.0,1.0219,1.2747,0
3,evening,wednesday,11.0,60.0,0.7867,948.0,1092.0,1584.0,1203.0,1.0219,1.2511,0
4,night,wednesday,11.2,59.6,0.7888,836.0,1205.0,1490.0,1110.0,0.7432,1.1565,0


In [5]:
training_data = data.drop(['true_label', 'true_error'], axis=1)
training_data.head(2)

,time_slot,day_of_week,T,RH,AH,PT08.S2(NMHC),PT08.S3(NOx),PT08.S4(NO2),PT08.S5(O3),given_label
0,evening,wednesday,13.6,48.9,0.7578,1046.0,1056.0,1692.0,1268.0,1.2366
1,evening,wednesday,13.3,47.7,0.7255,955.0,1174.0,1559.0,972.0,1.1747


In [ ]:
y = training_data['given_label']
X = training_data.drop(['given_label'], axis=1)

###### Add your training code here ###############
# model = 



# predictions = model.predict()

##### saving the predictions ####################
savepath = "./../predictions/model_predictions.npy"
np.save(savepath, np.array(predictions))

## Section section is relevant only if using AutoGluon

Uncomment the code in following cells and run to generate output and save to predictions folder. 

### set up arguments 

In [14]:
from autogluon.tabular import TabularDataset, TabularPredictor

data_without_label = training_data.drop(['given_label'], axis=1)
savepath = "./../trained_models/"
if not os.path.isdir(savepath):
    os.mkdir(savepath)

predictorArgs = {
                    "path"             : savepath, 
                    "label"            : 'given_label',
                    "eval_metric"      : 'r2',
                    "problem_type"     : 'regression'
                }

hyperparameters = { 
                    'GBM'   : {}, 
                    'FASTAI': {}, 
                    'RF'    : {'criterion': 'squared_error', 
                                'ag_args': {'name_suffix': 'MSE', 
                                'problem_types': ['regression', 'quantile']}}
                    }

### Normal Model fitting

In [15]:
predictor = TabularPredictor(**predictorArgs)
predictor.fit(training_data, hyperparameters=hyperparameters)
leaderboard = predictor.leaderboard(training_data, silent=True)
leaderboard

Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1615.19 MB
	Train Data (Original)  Memory Usage: 1.35 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerator...
	Stage 4 Generators:
		Fitting DropUniqueFeatureGenerator...
	Types of features in original data (raw dtype, special dtyp

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE,0.987238,0.939052,0.187330,0.037858,1.620574,0.187330,0.037858,1.620574,1,True,2
1,LightGBM,0.977422,0.949111,0.035244,0.008235,2.754683,0.035244,0.008235,2.754683,1,True,1
2,WeightedEnsemble_L2,0.969343,0.958884,0.101127,0.015792,6.474308,0.022028,0.000154,0.027975,2,True,4
3,NeuralNetFastAI,0.967854,0.958804,0.043855,0.007403,3.691650,0.043855,0.007403,3.691650,1,True,3


### Cross Validation fitting

In [16]:
predictor_cv = TabularPredictor(**predictorArgs) 
predictor_cv.fit(training_data, presets="best_quality", num_stack_levels= 0, hyperparameters=hyperparameters)
leaderboard_cv = predictor_cv.leaderboard(training_data, silent=True)
leaderboard_cv

Presets specified: ['best_quality']
Stack configuration (auto_stack=True): num_stack_levels=0, num_bag_folds=8, num_bag_sets=1
Beginning AutoGluon training ...
AutoGluon will save models to "./../trained_models/"
AutoGluon Version:  0.5.2
Python Version:     3.9.15
Operating System:   Darwin
Train Data Rows:    7344
Train Data Columns: 9
Label Column: given_label
Preprocessing data ...
Using Feature Generators to preprocess the data ...
Fitting AutoMLPipelineFeatureGenerator...
	Available Memory:                    1702.62 MB
	Train Data (Original)  Memory Usage: 1.35 MB (0.1% of available memory)
	Inferring data type of each feature based on column values. Set feature_metadata_in to manually specify special dtypes of the features.
	Stage 1 Generators:
		Fitting AsTypeFeatureGenerator...
	Stage 2 Generators:
		Fitting FillNaFeatureGenerator...
	Stage 3 Generators:
		Fitting IdentityFeatureGenerator...
		Fitting CategoryFeatureGenerator...
			Fitting CategoryMemoryMinimizeFeatureGenerat

,model,score_test,score_val,pred_time_test,pred_time_val,fit_time,pred_time_test_marginal,pred_time_val_marginal,fit_time_marginal,stack_level,can_infer,fit_order
0,RandomForestMSE_BAG_L1,0.992847,0.947426,0.352331,0.199404,1.944709,0.352331,0.199404,1.944709,1,True,2
1,WeightedEnsemble_L2,0.973641,0.962046,0.949344,0.287443,33.379785,0.001743,0.000246,0.053073,2,True,4
2,LightGBM_BAG_L1,0.970271,0.946488,0.176888,0.050563,16.014413,0.176888,0.050563,16.014413,1,True,1
3,NeuralNetFastAI_BAG_L1,0.969145,0.961762,0.595270,0.087793,31.382003,0.595270,0.087793,31.382003,1,True,3


### predicting and saving arrays 

In [17]:
models = {  'normal'            : list(leaderboard['model']), 
            'crossValidation'   : list(np.append(leaderboard_cv['model'], ['oof']))}

predictors = { 'normal'          : predictor, 
               'crossValidation' : predictor_cv}

# check if path is available, else create it. 
pred_path = "./../predictions/"
if not os.path.isdir(pred_path):
    os.mkdir(pred_path)

for model_type in models.keys():
    if model_type == "normal": 
        for model in models[model_type]:
            savepath = os.path.join(pred_path, model+".npy")
            predictions = predictors[model_type].predict(data_without_label, model=model)
            np.save(savepath, np.array(predictions))
    elif model_type == "crossValidation":
        for model in models[model_type]:
            if model == "oof":
                predictions = predictors[model_type].get_oof_pred()
                savepath = os.path.join(pred_path, model+".npy")
            elif model in models['normal']:
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+"_CV.npy")
            else: 
                predictions = predictors[model_type].predict(data_without_label, model=model)
                savepath = os.path.join(pred_path, model+".npy")

            np.save(savepath, np.array(predictions))

[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNative.cpp:229] Warning: Cannot set number of intraop threads after parallel work has started or after set_num_threads call when using native parallel backend (function set_num_threads)
[W ParallelNati